# Google auto-suggestions request

### Step 1

After having the top 20 most vistied cities, we can get auto suggestions for each city from google to see what stereotypes people have about that city using the requests library. We use two queries in this function to make the result richer.

**TIP:**
 'Hong Kong SAR' is a term that emphasizes the administrative characteristic of the city, and it is rarely used in everyday life. Therefore, it is changed into 'Hong Kong' in this function to suit the spoken language.

In [1]:
import requests

def get_auto_suggestions(city):
    adjusted_city = 'Hong Kong' if city == 'Hong Kong SAR' else city
    queries = [f"why is {adjusted_city} so", f"why is {adjusted_city} always"]
    
    all_suggestions = []
    for query in queries:
        url = f"https://www.google.com/complete/search?q={query}&client=firefox"
        response = requests.get(url)
        if response.status_code == 200:
            suggestions = response.json()[1]
            all_suggestions.extend(suggestions)
        else:
            print(f"Fail: {query}")
    return all_suggestions

This returns a list of simple sentences that needs to be processed. Here is the example of London:

In [3]:
get_auto_suggestions('London')

['why is london so expensive',
 'why is london so dangerous',
 'why is london so popular',
 'why is london so big',
 'why is london so foggy',
 'why is london so busy today',
 'why is london so dirty',
 'why is london so cold',
 'why is london so cloudy',
 'why is london so rainy',
 'why is london always cloudy',
 'why is london always rainy',
 'why is london always cold',
 'why is london always foggy',
 'why is london always grey',
 'why is london always raining',
 'why is london always so cloudy',
 'why is london always windy',
 'why is london always gray',
 'why is london always hotter']

### Step 2

We then need to scrape all the 20 cities and extract the discriptive words in such lists which makes the output neat and clean. In this function, the re library is used to locate the position of those discriptive words.

In [ ]:
import re

def extract_words(cities):
    city_stereotype = {}
    for city in cities:
        suggestions = get_auto_suggestions(city)
        stereotype_list = [] 
        for suggestion in suggestions:
            if 'why' in suggestion.lower() and ('so' in suggestion.lower() or 'always' in suggestion.lower()):
                suggestion_parts = {}
                for delimiter in ['so', 'always']:
                    suggestion_parts[delimiter] = re.split(fr'\b{delimiter}\b', suggestion)
                    if len(suggestion_parts[delimiter]) > 1:
                        stereotype = suggestion_parts[delimiter][1].strip()
                        stereotype_list.append(stereotype)
        full_stereotype_str = ', '.join(stereotype_list)
        city_stereotype[city] = full_stereotype_str
    return city_stereotype

### Step 3

In [ ]:
def filter_weather_words(suggestion_dict):
    weather_words = ["sunny", "rainy", "windy", "cloudy", "foggy", "hot", "cold", "stormy", "humid", "dry", "wet"]
    new_dict = {}
    for key, words_str in suggestion_dict.items():
        words_list = words_str.split(', ')
        filtered_word_list = [word for word in words_list if word in weather_words]
        new_dict[key] = filtered_word_list
    return new_dict